# Análisis Regional con Estilo The Economist
Este cuaderno explora la evolución mensual del comercio regional utilizando visualizaciones inspiradas en el estilo de *The Economist*.

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path

In [3]:
project_root = Path.cwd()
if not (project_root / 'data').exists():
    project_root = project_root.parent

DATA_PATH = project_root / 'data/processed/ventas_regionales_notebook.csv'
if not DATA_PATH.exists():
    raise FileNotFoundError(f'No se encontró el archivo {DATA_PATH}. Ejecuta el pipeline antes de continuar.')

df = pd.read_csv(DATA_PATH, parse_dates=['periodo'])
df.sort_values(['region', 'periodo'], inplace=True)
df.head()

,region,periodo,anio,mes,trimestre,monto_total,monto_promedio,observaciones
0,Araucania,2018-01-01,2018,1,2018Q1,359,359.0,1
1,Araucania,2018-02-01,2018,2,2018Q1,404,404.0,1
2,Araucania,2018-03-01,2018,3,2018Q1,410,410.0,1
3,Araucania,2018-04-01,2018,4,2018Q2,402,402.0,1
4,Araucania,2018-05-01,2018,5,2018Q2,394,394.0,1


In [4]:
# Métricas agregadas por región y trimestre para contextualizar tendencias.
regional_summary = (
    df.groupby(['region', 'anio'])
      .agg(monto_total_anual=('monto_total', 'sum'),
           monto_promedio_anual=('monto_promedio', 'mean'))
      .reset_index()
)
regional_summary.head()

,region,anio,monto_total_anual,monto_promedio_anual
0,Araucania,2018,4680,390.000000
1,Araucania,2019,4956,413.000000
2,Araucania,2020,5164,430.333333
3,Araucania,2021,6627,552.250000
4,Araucania,2022,7602,633.500000


In [6]:
# Serie temporal de montos totales por región.
fig = px.line(
    df,
    x='periodo',
    y='monto_total',
    color='region',
    markers=True,
    title='Ventas Mensuales por Región'
)
fig.update_layout(
    font=dict(family='Georgia, serif', color='#1e293b'),
    title=dict(font=dict(size=18, family='Georgia, serif', color='#1e293b')),
    xaxis=dict(
        title=dict(font=dict(family='Georgia, serif')),
        tickfont=dict(family='Georgia, serif')
    ),
    yaxis=dict(
        title=dict(text='Monto total (MM CLP)', font=dict(family='Georgia, serif')),
        tickfont=dict(family='Georgia, serif')
    ),
    legend=dict(font=dict(family='Georgia, serif')),
    plot_bgcolor='#ffffff',
    paper_bgcolor='#ffffff'
)
fig.update_traces(line=dict(width=2.2))
fig

In [7]:
# Variación interanual por región.
growth = df.copy()
growth['monto_total_lag_12'] = growth.groupby('region')['monto_total'].shift(12)
growth = growth.dropna(subset=['monto_total_lag_12']).copy()
growth['variacion_interanual'] = ((growth['monto_total'] / growth['monto_total_lag_12']) - 1) * 100
fig_growth = px.line(
    growth,
    x='periodo',
    y='variacion_interanual',
    color='region',
    markers=True,
    title='Variación interanual de las ventas regionales (%)'
 )
fig_growth.update_layout(
    font=dict(family='Georgia, serif', color='#1e293b'),
    title=dict(font=dict(size=18, family='Georgia, serif', color='#1e293b')),
    xaxis=dict(
        title=dict(font=dict(family='Georgia, serif')),
        tickfont=dict(family='Georgia, serif')
    ),
    yaxis=dict(
        title=dict(text='Variación interanual (%)', font=dict(family='Georgia, serif')),
        tickfont=dict(family='Georgia, serif')
    ),
    legend=dict(font=dict(family='Georgia, serif')),
    plot_bgcolor='#ffffff',
    paper_bgcolor='#ffffff'
 )
fig_growth.update_traces(line=dict(width=2.2))
fig_growth.add_hline(y=0, line_dash='dot', line_color='#94a3b8')
fig_growth